In [2]:
from tqdm import tqdm

In [3]:
#Cntrl+Enter выполнить ячейку

#Путь к файлу экспорта из Pulseview
fname = "../beckhoff-sniff/13/txt-dup"

f = open(fname, 'r')
lines = f.readlines()
lines = lines[2:]
print("1st 8 lines:")
for i in range(20):
    print(lines[i])

1st 8 lines:
D0:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D1:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D2:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D3:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D4:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D5:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D6:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D7:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D0:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D1:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D2:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D3:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111

D4:11111111 11111111 11111111 11111111 11111111 11111111 11111111 11111111


In [4]:
#Тут надо ввести номер канала, который ты хочешь расшифровывать. В файле 12MHZ это 1, например 
channel = 2


In [11]:
print('abc', end='')

print('ee')

ee


In [ ]:
#Вычленяем нужный канал, удаляем пробелы, переносы строки и т п
#работает долго, секунд пять, т к файл большой

dump = ""
i = 0

for line in lines[0:-1]:
    print(i, ' of ', len(lines), '\r', end="")
    i = i + 1;
    if line[0:3] == "D{}:".format(channel) or line[0:4] == "D{}:".format(channel):
        dump = dump + line.replace("\n"," ")
dump = dump.replace(" ","")
dump = dump.replace("D{}:".format(channel),"")
print(dump[0:100])

497  of  14642240 

57108  of  14642240 4642240  14642240    of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240  14642240  of  14642240   of  14642240  14642240 

In [ ]:
#находим начало синхронизирующей частоты. По сути - выбрасываем префикс из одинаковых символов

sync_pos = 0
a = dump[0]
while(dump[sync_pos] == a):
    sync_pos = sync_pos + 1
dump = dump[sync_pos:]
print(dump[0:100])

In [ ]:
#частота захвата в мегагерцах
freq = 12


#Хавает на вход длинк последовательности одинаковых символов, отдает ее декод из nrzi 
def parse(s):
    if(s < 3):
        return []
    if(s%(freq//2) < (freq//4)):
        x = 2*s//freq
    else:
        x = (2*s//freq) + 1
    ans = [False]
    ans.extend([True for i in range(0,x-1)])
    return ans

parse(2), parse(3), parse(8), parse(9)

In [ ]:
# разбираем NRZ-I, преобразовываем X МГЦ в 2 МГЦ, выкидываем несущую частоту
# работает долго, секнд 20 для 12MHZ.txt


# идем по дампу.несущая частота - это ~48 нулей, затем ~48 единиц, и т д.(для 12 мгц, для произвольной - 4*freq)
# В посылках нет 8 повторяющихся так что последовательности длины 45    (4*freq - freq/4 в общем случае) 
# то что длины 45 и больше - точно несущая частота
sync_pos = 0
a = dump[0]
last_pos = 0
bitdump = []
while sync_pos < len(dump):
    a = dump[last_pos]
    sync_pos = last_pos
    while(sync_pos < len(dump) and dump[sync_pos] == a):
        sync_pos = sync_pos + 1
    if(sync_pos - last_pos <= 4*freq - freq//4):
        bitdump.extend(parse(sync_pos-last_pos))
    last_pos = sync_pos
    
print(bitdump[0:16])

#bd - то же что и bitdump, только не булевый true/false, а 0/1 
bd = ["1" if bitdump[i] else "0" for i in range(0,len(bitdump))]
bd_str = "".join(bd)
print(bd_str[0:100])

In [ ]:
len(bd_str)

In [ ]:
RAW = []
ENC = []
total = 0
ff = open('/home/arabella/dump0', 'wb')
cycles_short = 0
cycles_long = 0
syncro_stage = 0

In [ ]:
# Это часть недописанная, и я не до конца уверен в том, что она работает правильно.
# Разделяет вход на посылки, умеет часть и них классифицировать и парсить.
# Предполагается, что мы начли захватывать до начала посылок


pos = 0
i = 0
debug_lvl = 1 #1 - выводить сырое сообщение, 0 - нет
cycle_count = 0;
phaze = 0;
drv_done=0
DECODE_BS = True

# Это устанавливается в настройках мастера, в твинкэте типа
drive_num = 3 # ожидаемое число приводов. 

#число активных выходных IDN в каждом приводе. Список есть в S-0-15/S-0-16, насколько помню.
ind_num = {1:1,2:1, 3:1} #{номер привода: число идн}, например {2:2, 3:1, 1:2, 4:2}
y = 0


def fefind(start):
    while(len(bd_str) > start + 8 and bd_str[start : start+8] != '01111110'):
        start = start + 1
    if len(bd_str) <= start+8:
        start = -1
    return start



while True:
    #Ищем два последовательных 7Е, считаем что между ними - посылка x, y - позиции начала этих 7Е
#     x = bd_str.find("01111110",pos, len(bd_str))
    x = fefind(pos)
    if x == -1:
        break
    y = fefind(x+1)
#     y = bd_str.find("01111110",x + 1, len(bd_str))
    if y == -1:
        break
    #Отбрасываем слишком короткие бессмысленные посылки, в которых меньше 3 байт данных 
    #(в любой посылке должен быть один байт адреса и два байта FC)
#     pos = y + 1
    sss = bd_str[x+8:y]
    if(len(sss) < 24):
#         print('trash: ',bd_str[x+8:y])
        pos = y
        continue
    pos = y

#         z = fefind(y + 1)
#         if z == -1:
#             break
#         if z - y > y - x:
#             x = y
#             y = z
#             pos = x + 1
#             print('trash: ', bd_str[oldy:x])
#         else:
#             if(debug_lvl == 1):
#                 print("\n\n\n \033[91mToo short message. RAW DATA: '\033[0m'",bd_str[x:y+8], "start at ", x)

#     elif(len(sss) > 40*8):
#         z = bd_str.find("01111110",y + 1, len(bd_str))
#         if z == -1:
#             break
#         if z - y > y - x:
#             x = y
#             y = z
#             pos = z + 1
#         else:
#             if(debug_lvl == 1):
#                 print("\n\n\n \033[91mToo BIG message. RAW DATA: '\033[0m'",bd_str[x:y+8], y+ 8 -x)
    total = total + 1
#     if(total == 70):
#         print("\n\n\n \033[91m!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#     print('\n\n')
#     print('pos',x)
#     print('raw:      ', bd_str[x:y+8])
    sss = bd_str[x:y+8];
    char = '0'
    b = ''
    for i in sss:
        if i == '0' and char == '0':
            char = '1' 
        elif i == '0' and char == '1':
            char = '0'
        b = b + char
    additional_bits_length = 2;
    b = '0'* additional_bits_length + b + b[-1] * additional_bits_length
    if(len(b)%8 > 0):
        b = b + b[-1]*(8 - (len(b)%8))
    print('nrzi enc  ', b)
    
    raw = [b[8*v:8*v+8] for v in range(0, len(b) // 8)]



    #Убираем нолики, которые протокол вставляет после каждых 5 единиц('bit-stuffing')-
    if(DECODE_BS):
        if(len(sss) // 8 > 10):
            print('bs decoded', sss)
        ssss = [sss[8*v:8*v+8] for v in range(0, len(sss) // 8)]
      
    print(len(b))
    print(total)
    RAW.append( str(format(len(b) // 8, 'b')) )
    if(len(ssss) < 9):
        if(int(ssss[2],2) == 0x00):
            syncro_stage = 0
        if(int(ssss[2],2) == 0x80):
            syncro_stage = 10
        if(int(ssss[2],2) == 0xC0):
            syncro_stage = 20
        if(int(ssss[2],2) == 0xA0):
            syncro_stage = 30
        if(int(ssss[2],2) == 0xE0):
            syncro_stage = 34
        if(int(ssss[2],2) == 0xF0):
            syncro_stage = 40
        if(syncro_stage < 34):
            cycles_short = cycles_short + 1
        else:
            cycles_long = cycles_long + 1
        RAW.append( bin(syncro_stage)[2:] )   

    print('sinc stage', syncro_stage, bin(syncro_stage)[2:])
    
    
   
    RAW.extend( raw )

    ENC.append( str(format(len(ssss), 'b')) )
    ENC.extend( ssss )
        #sss - массив из строк по 8 символов ака байтов
#        try:
#             #Счетчик числа сказавших свое приводов
            
#             if(debug_lvl == 1):
#                 print("\n\n\nRAW: ", " ".join(sss))
#             else:
#                 print('\n')
                
#             # MST всегда 6 байтов, адрес - 6 единиц
#             if(sss[1] == '11111111' and len(sss) == 6):
#                 if(phaze > 1 and drv_done < drive_num):
#                     print("\033[91m WARNING: not all drive replayed prevous cycle \033[0m")
#                     if(phaze < 4):
#                         print("\033[91m Parsing may be incorrect \033[0m")
#                 if(phaze > 1 and drv_done < drive_num + 1):
#                     print("\033[91m WARNING: no MDT or one DT's missing prevous cycle \033[0m")
#                     if(phaze < 4):
#                         print("\033[91m Parsing may be incorrect \033[0m")
#                 print("\n\n\n========  Cycle# %d ======== start %d stop %d" % (cycle_count, x , y+8))
#                 print("MST: communication phaze", int(sss[2][::-1],2) )
#                 cycle_count = cycle_count + 1                 
#                 drv_done = 0
#                 phaze = int(sss[2][::-1],2)
#                 log.append(str(format(6, 'b')))
#                 log.extend(sss)
#                 total = total + 1
#                 continue
                
#             # DT не может быть отправлена всем сразу всем, ее адрес - не 11111111
#             # В документации написано: В фазах 1 и 2 мастер "шлет индивидуальные MDT для каждого привода"
#             # Я что это означает в плане поля "адрес" я так и не понял.
            
#             if(sss[1] != '11111111'):
#                 if(phaze < 3):
#                     print('Some telegram')
#                     if(phaze > 1):
#                         if(drv_done < drive_num):
#                             print("Looks like It's DT")
#                             drv_done += 1
#                         else:
#                             print("Looks like It's individual MDT")
#                             drv_done += 1
                        
#                     for i in range(len(sss)):
#                         print(sss[i], end = ' ')
#                     continue
#                 # если фаза хотя-бы третья, то можно однозначто считать что то что имеет адрес не 1 - это DT
#                 # и она как-то парсится
#                 print("DT: \n    Address: ", int(sss[1][::-1],2))
#                 print("    Drive Status Word: ", sss[2],sss[3])
#                 print("    Service channel:   ", sss[4],sss[5])
#                 print("    IDNS: ")
#                 drv_done += 1
#                 # Первые 5 последние 3 байта - служебные. 4 Байта на каждую IDN 
#                 # (некоторым вроде надо только 2, надо смотреть в документации к приводам на конкретную IDN)
#                 # Но я 2-х байтовых не встречал
                
#                 idn_start = 6
#                 while idn_start < len(sss) - 3:
#                     value = sss[idn_start] + sss[idn_start+1] + sss[idn_start+2] + sss[idn_start+3]
#                     idn_start = idn_start + 4
#                     #переворачиваем значение (т.к. little-endian) и выводим
#                     print("        ",int(value[::-1],2))
            
#             # Парсер MDT не написан от слова совсем, просто выводит то что классифицируется как 
#             # MDT по остаточному принципу
#             else:
#                 drv_done += 1
#                 offset = 2
#                 print("MDT:")
#                 log.append( str(format(len(sss), 'b')) )
#                 log.extend( sss )
#                 total = total + 1
#                 for i in range(1,drive_num + 1):
#                       print('    Drive #%d'% i)
#                       print('    Master Control Word: ', sss[offset], sss[offset + 1])
#                       offset += 2
#                       print('    Service Channel:     ', sss[offset], sss[offset + 1])
#                       offset += 2
#                       print("    IDNS:")
#                       for idnN in range(ind_num[i]):
#                           value = sss[offset] + sss[offset+1] + sss[offset+2] + sss[offset+3]
#                           offset += 4
#                           #переворачиваем значение (т.к. little-endian) и выводим
#                           print("        ",int(value[::-1],2))

            
                
#         except Excepion as e:
#             print("eror during parsing:", e)
#             continue

In [ ]:
print(cycles_short, cycles_long)

In [ ]:
total

In [ ]:
int_RAW = [total // 256, total%256]
int_ENC = [total // 256, total%256]

int_RAW.extend([int(x, 2) for x in RAW])
int_ENC.extend([int(x, 2) for x in ENC])


int_RAW[:100]

In [ ]:
len('011111101000000010000000000000000000000000000000000100010001101101111110')

In [ ]:
byteraw = bytearray(int_RAW)
byteenc = bytearray(int_ENC)

In [ ]:

raw = open("byte-raw.bin", "wb")
dec = open("byte-dec.bin", "wb")
# write to file
raw.write(byteraw)
dec.write(byteenc)
